In [109]:
%load_ext rpy2.ipython
import rpy2.robjects.lib.ggplot2 as ggplot2
import childespy
import numpy as np
import os
import imp
import pandas as pd
import transformers
import torch
import scipy.stats
import copy
from string import punctuation
import transfomers_bert_completions
imp.reload(transfomers_bert_completions)

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


<module 'transfomers_bert_completions' from '/home/stephan/notebooks/child-directed-listening/transfomers_bert_completions.py'>

# Masked Language Prediction Softmax with BERT 

## Older models

In [110]:
from pytorch_pretrained_bert import BertForMaskedLM
from transformers import BertTokenizer

In [111]:
cmu_in_childes = pd.read_csv('phon/cmu_in_childes.csv')
cmu_2syl_inchildes = cmu_in_childes.loc[cmu_in_childes.num_vowels <=2]

In [320]:
adult_bertMaskedLM = BertForMaskedLM.from_pretrained('bert-large-uncased')
adult_bertMaskedLM.eval()
adult_tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')
adult_softmax_mask, adult_vocab = transfomers_bert_completions.get_softmax_mask(adult_tokenizer, cmu_2syl_inchildes.word)

#
ft1_bertMaskedLM = BertForMaskedLM.from_pretrained('model_output')
ft1_bertMaskedLM.eval()
ft1_tokenizer = BertTokenizer.from_pretrained('model_output')
ft1_softmax_mask, ft1_vocab = transfomers_bert_completions.get_softmax_mask(ft1_tokenizer, cmu_2syl_inchildes.word)


ft2_bertMaskedLM = BertForMaskedLM.from_pretrained('model_output2')
ft2_bertMaskedLM.eval()
ft2_tokenizer = BertTokenizer.from_pretrained('model_output2')
ft2_softmax_mask, ft2_vocab = transfomers_bert_completions.get_softmax_mask(ft2_tokenizer, cmu_2syl_inchildes.word)

### Adult Model

In [121]:
imp.reload(transfomers_bert_completions)
test= transfomers_bert_completions.compare_completions("mommy [MASK] .", 
        adult_bertMaskedLM, adult_tokenizer, adult_softmax_mask)
test[1]

['[CLS]', 'mommy', '[MASK]', '.', '[SEP]']


,prob,word,rank
83,1.234673e-01,said,0
592,1.197654e-01,says,1
2943,6.757364e-02,screamed,2
4552,4.424965e-02,screams,3
1510,4.048109e-02,knows,4
...,...,...,...
4308,9.112945e-09,hire,7899
4387,8.820145e-09,pupil,7900
5744,8.202045e-09,fencing,7901
7465,4.160909e-09,blooded,7902


In [122]:
transfomers_bert_completions.compare_completions("where [MASK] ?", adult_bertMaskedLM,
    adult_tokenizer, adult_softmax_mask)[1]

['[CLS]', 'where', '[MASK]', '?', '[SEP]']


,prob,word,rank
29,6.473997e-01,to,0
643,2.112121e-01,else,1
107,3.109264e-02,now,2
86,1.295018e-02,then,3
41,1.068886e-02,at,4
...,...,...,...
6355,4.944750e-10,jays,7899
7866,4.470875e-10,condor,7900
7220,2.171247e-10,spitting,7901
7037,1.800662e-10,alfa,7902


In [124]:
transfomers_bert_completions.compare_completions("hi [MASK] .", 
    adult_bertMaskedLM, adult_tokenizer, adult_softmax_mask)[1]

['[CLS]', 'hi', '[MASK]', '.', '[SEP]']


,prob,word,rank
9,3.663202e-02,j,0
652,2.526271e-02,dr,1
6,2.271786e-02,g,2
562,2.210053e-02,mr,3
13,2.103107e-02,n,4
...,...,...,...
5008,6.478103e-08,spit,7899
6482,6.150631e-08,drown,7900
4904,3.894868e-08,drowned,7901
7420,3.889998e-08,writ,7902


In [126]:
transfomers_bert_completions.compare_completions("what [MASK] .", adult_bertMaskedLM,
    adult_tokenizer, adult_softmax_mask)[1]

['[CLS]', 'what', '[MASK]', '.', '[SEP]']


,prob,word,rank
91,4.661836e-01,if,0
775,3.032369e-01,happened,1
25,3.079754e-02,the,2
107,2.106155e-02,now,3
2592,1.707017e-02,happens,4
...,...,...,...
5969,2.850364e-09,watkins,7899
6780,2.795745e-09,teller,7900
7037,2.744674e-09,alfa,7901
6821,1.950509e-09,milky,7902


In [131]:
completions = transfomers_bert_completions.compare_completions("go [MASK] .", 
    adult_bertMaskedLM, adult_tokenizer, adult_softmax_mask)[1]
completions

['[CLS]', 'go', '[MASK]', '.', '[SEP]']


,prob,word,rank
1237,2.241399e-01,ahead,0
35,1.854041e-01,on,1
190,6.621100e-02,home,2
187,3.992473e-02,away,3
93,3.229499e-02,back,4
...,...,...,...
5358,3.414107e-08,crushing,7899
7263,2.671358e-08,cords,7900
6986,2.521703e-08,terence,7901
7342,2.447127e-08,tending,7902


In [132]:
transfomers_bert_completions.bert_completions("It's time to go to the [MASK] .", adult_bertMaskedLM,
    adult_tokenizer, np.array(range(len(adult_softmax_mask)) ))

['[CLS]', 'it', "'", 's', 'time', 'to', 'go', 'to', 'the', '[MASK]', '.', '[SEP]']


(array([1.4456957e-07, 1.4368452e-07, 1.6101605e-07, ..., 3.6886934e-07,
        2.9436049e-07, 9.5527675e-08], dtype=float32),
               prob      word  rank
 5723  1.267168e-01  bathroom     0
 2902  7.437576e-02  hospital     1
 3509  5.318297e-02     beach     2
 5691  3.887066e-02    movies     3
 3460  2.636691e-02    doctor     4
 ...            ...       ...   ...
 1104  7.018393e-09         ı  7899
 1577  6.265811e-09         ℓ  7900
 5605  5.924017e-09     ##hip  7901
 1537  5.464219e-09         ⁱ  7902
 1105  4.721904e-09         ł  7903
 
 [7904 rows x 3 columns])

# BERT without Context

In [278]:
all_tokens_phono = pd.read_pickle('csv/pvd_utt_glosses_phono_cleaned_inflated.pkl')

In [237]:
imp.reload(transfomers_bert_completions)
_,predictions =  transfomers_bert_completions.bert_completions(
    "[MASK] nonfiction book had a call number on its spine", adult_bertMaskedLM, adult_tokenizer, 
    adult_softmax_mask)
predictions.head(10)

['[CLS]', '[MASK]', 'nonfiction', 'book', 'had', 'a', 'call', 'number', 'on', 'its', 'spine', '[SEP]']
[3.6191159e-01 1.4366881e-05 5.9900581e-06 ... 1.3998201e-08 1.2068814e-08
 7.0058026e-09]
['a' 'b' 'c' ... 'hideout' 'pudding' 'stalks']


,prob,word,rank
0,0.361912,a,0
175,0.297483,each,1
159,0.110310,any,2
25,0.101745,the,3
80,0.078346,no,4
51,0.022545,this,5
39,0.005070,his,6
54,0.002908,my,7
37,0.002811,that,8
63,0.002086,all,9


In [238]:
# otsb: off the shelf BERT
_,predictions =  transfomers_bert_completions.bert_completions(
    "what an excellent plan [MASK]", adult_bertMaskedLM, adult_tokenizer, adult_softmax_mask)
predictions.head(10)

['[CLS]', 'what', 'an', 'excellent', 'plan', '[MASK]', '[SEP]']
[4.6391021e-03 2.6643262e-04 4.5387544e-05 ... 3.8616272e-05 4.1947722e-05
 1.9335086e-05]
['a' 'b' 'c' ... 'hideout' 'pudding' 'stalks']


,prob,word,rank
27,0.198977,and,0
148,0.054641,because,1
37,0.036092,that,2
34,0.031438,for,3
25,0.026816,the,4
49,0.025527,but,5
33,0.018605,as,6
51,0.010626,this,7
107,0.009459,now,8
54,0.009208,my,9


In [239]:
# otsb: off the shelf BERT
_,predictions =  transfomers_bert_completions.bert_completions(
    "what an excellent [MASK] .", adult_bertMaskedLM, adult_tokenizer, adult_softmax_mask)
predictions.head(10)


['[CLS]', 'what', 'an', 'excellent', '[MASK]', '.', '[SEP]']
[1.2686984e-05 5.9214440e-06 3.2423511e-06 ... 4.4299151e-05 3.3103188e-05
 2.0205528e-07]
['a' 'b' 'c' ... 'hideout' 'pudding' 'stalks']


,prob,word,rank
847,0.098999,question,0
703,0.089799,plan,1
165,0.058697,man,2
1111,0.040538,choice,3
595,0.028590,friend,4
232,0.024965,thought,5
162,0.022737,day,6
382,0.020267,woman,7
809,0.014959,job,8
392,0.013705,story,9


In [240]:
# no context, otsb: off the shelf BERT
priors, completions, stats = transfomers_bert_completions.get_stats_for_failure(
    all_tokens_phono, 16764425, adult_bertMaskedLM, adult_tokenizer, adult_softmax_mask,
    None, use_speaker_labels=False)

[5.7512279e-05 4.6954942e-06 1.7185309e-06 ... 1.9388933e-06 7.3790943e-06
 2.9924431e-07]
['a' 'b' 'c' ... 'hideout' 'pudding' 'stalks']


/home/stephan/notebooks/child-directed-listening/lib/python3.7/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [241]:
print(priors) 
print(completions)
print(stats)

[5.7512279e-05 4.6954942e-06 1.7185309e-06 ... 1.9388933e-06 7.3790943e-06
 2.9924431e-07]
              prob      word  rank
179   7.037868e-01     place     0
429   6.678923e-02     thing     1
165   1.722324e-02       man     2
823   1.432071e-02       guy     3
5387  1.370183e-02  nonsense     4
...            ...       ...   ...
6446  1.696233e-09  forwards  7899
7072  1.578553e-09   graders  7900
4302  1.565044e-09      laps  7901
6001  1.560177e-09     wills  7902
4745  8.996724e-10    thirds  7903

[7904 rows x 3 columns]
       rank  prob   entropy  num_tokens_in_context  bert_token_id
26826   NaN   NaN  2.876822                      4          26826


In [242]:
# no context, otsb: off the shelf BERT
transfomers_bert_completions.get_stats_for_success(all_tokens_phono, 16759315, adult_bertMaskedLM, 
        adult_tokenizer, adult_softmax_mask)

[1.5922917e-02 1.4876368e-02 1.5768111e-02 ... 1.0458241e-05 4.0509703e-06
 3.4971272e-06]
['a' 'b' 'c' ... 'hideout' 'pudding' 'stalks']


(array([[1.5922917e-02, 1.4876368e-02, 1.5768111e-02, ..., 1.0458241e-05,
         4.0509703e-06, 3.4971272e-06]], dtype=float32),
 [              prob      word  rank
  15    1.550260e-01         p     0
  1335  7.418367e-02        ed     1
  2720  2.585679e-02        op     2
  0     1.592292e-02         a     3
  2     1.576811e-02         c     4
  ...            ...       ...   ...
  3858  2.001719e-07   smelled  7899
  4844  1.871311e-07    tasted  7900
  4427  1.673072e-07    blamed  7901
  7190  1.354257e-07   blaming  7902
  5984  1.214368e-07  highness  7903
  
  [7904 rows x 3 columns]],
     rank      prob   entropy  num_tokens_in_context  bert_token_id  \
 42  7113  0.000002  8.392303                      1             42   
 
     mask_position  token  utterance_id  
 42              0  mommy      16759315  )

In [243]:
# Fine-tuned BERT: don't use speaker labels
transfomers_bert_completions.bert_completions(
    "[MASK] .", ft1_bertMaskedLM, ft1_tokenizer, adult_softmax_mask)

['[CLS]', '[MASK]', '.', '[SEP]']
[1.1490965e-02 2.0865996e-03 1.2678547e-03 ... 3.6077824e-06 1.9368913e-04
 7.7008317e-06]
['a' 'b' 'c' ... 'hideout' 'pudding' 'stalks']


(array([1.1490965e-02, 2.0865996e-03, 1.2678547e-03, ..., 3.6077824e-06,
        1.9368913e-04, 7.7008317e-06], dtype=float32),
               prob     word  rank
 25    1.754317e-02      the     0
 80    1.313224e-02       no     1
 0     1.149097e-02        a     2
 38    1.031996e-02       it     3
 35    9.670383e-03       on     4
 ...            ...      ...   ...
 7663  5.419910e-07    boxed  7899
 7465  4.390227e-07  blooded  7900
 5448  4.197854e-07   billed  7901
 5146  3.850632e-07    posed  7902
 4542  3.315166e-07    petty  7903
 
 [7904 rows x 3 columns])

In [244]:
# no context ft1: Fine-tuned BERT: don't use speaker labels
transfomers_bert_completions.get_stats_for_failure(
    all_tokens_phono, 16764425, ft1_bertMaskedLM, ft1_tokenizer, ft1_softmax_mask,
    None, use_speaker_labels=False)

[1.2034437e-02 9.0256333e-04 1.4439654e-04 ... 5.4023002e-07 2.6916116e-05
 6.8373361e-06]
['a' 'b' 'c' ... 'hideout' 'pudding' 'stalks']


/home/stephan/notebooks/child-directed-listening/lib/python3.7/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


(array([1.2034437e-02, 9.0256333e-04, 1.4439654e-04, ..., 5.4023002e-07,
        2.6916116e-05, 6.8373361e-06], dtype=float32),
               prob      word  rank
 56    9.787571e-02       one     0
 429   8.345397e-02     thing     1
 32    5.292996e-02        is     2
 185   4.436808e-02      here     3
 34    3.257630e-02       for     4
 ...            ...       ...   ...
 5126  6.021823e-09  airports  7899
 7241  6.009501e-09   braking  7900
 7866  3.241802e-09    condor  7901
 5583  2.310708e-09   motions  7902
 5679  1.696319e-09    leaked  7903
 
 [7904 rows x 3 columns],
        rank  prob  entropy  num_tokens_in_context  bert_token_id
 26826   NaN   NaN  7.68566                      4          26826)

In [245]:
# no context, ft1: Fine-tuned BERT: don't use speaker labels
transfomers_bert_completions.get_stats_for_success(all_tokens_phono, 16759315, ft1_bertMaskedLM, 
        ft1_tokenizer, ft1_softmax_mask)

[1.1490965e-02 2.0865996e-03 1.2678547e-03 ... 3.6077824e-06 1.9368913e-04
 7.7008317e-06]
['a' 'b' 'c' ... 'hideout' 'pudding' 'stalks']


(array([[1.1490965e-02, 2.0865996e-03, 1.2678547e-03, ..., 3.6077824e-06,
         1.9368913e-04, 7.7008317e-06]], dtype=float32),
 [              prob     word  rank
  25    1.754317e-02      the     0
  80    1.313224e-02       no     1
  0     1.149097e-02        a     2
  38    1.031996e-02       it     3
  35    9.670383e-03       on     4
  ...            ...      ...   ...
  7663  5.419910e-07    boxed  7899
  7465  4.390227e-07  blooded  7900
  5448  4.197854e-07   billed  7901
  5146  3.850632e-07    posed  7902
  4542  3.315166e-07    petty  7903
  
  [7904 rows x 3 columns]],
     rank      prob    entropy  num_tokens_in_context  bert_token_id  \
 42   723  0.000264  10.683244                      1             42   
 
     mask_position  token  utterance_id  
 42              0  mommy      16759315  )

In [246]:
import transfomers_bert_completions
imp.reload(transfomers_bert_completions)

<module 'transfomers_bert_completions' from '/home/stephan/notebooks/child-directed-listening/transfomers_bert_completions.py'>

In [247]:
# ft2: Fine-tuned BERT: with speaker labels
transfomers_bert_completions.bert_completions(
    "[chi] [MASK] .", ft2_bertMaskedLM, ft2_tokenizer, ft2_softmax_mask)

['[CLS]', '[chi]', '[MASK]', '.', '[SEP]']
[7.1630292e-03 1.6241993e-03 1.0531463e-03 ... 1.5241826e-06 1.6998670e-04
 2.9817779e-06]
['a' 'b' 'c' ... 'hideout' 'pudding' 'stalks']


(array([7.1630292e-03, 1.6241993e-03, 1.0531463e-03, ..., 1.5241826e-06,
        1.6998670e-04, 2.9817779e-06], dtype=float32),
               prob        word  rank
 80    2.959480e-02          no     0
 630   1.779688e-02          oh     1
 73    1.736975e-02       there     2
 56    1.563328e-02         one     3
 988   1.324041e-02        yeah     4
 ...            ...         ...   ...
 5984  5.000543e-08    highness  7899
 1195  4.420516e-08      latter  7900
 7100  3.975406e-08     boasted  7901
 7334  3.718563e-08  yourselves  7902
 7541  1.118408e-08     parades  7903
 
 [7904 rows x 3 columns])

In [248]:
# ft2: Fine-tuned BERT: with speaker labels
transfomers_bert_completions.bert_completions(
    "[cgv] [MASK] .", ft2_bertMaskedLM, ft2_tokenizer, ft2_softmax_mask)

['[CLS]', '[cgv]', '[MASK]', '.', '[SEP]']
[4.4205962e-03 2.5260656e-03 1.5678611e-03 ... 1.1706089e-06 8.7236323e-05
 1.4746232e-05]
['a' 'b' 'c' ... 'hideout' 'pudding' 'stalks']


(array([4.4205962e-03, 2.5260656e-03, 1.5678611e-03, ..., 1.1706089e-06,
        8.7236323e-05, 1.4746232e-05], dtype=float32),
               prob        word  rank
 80    1.917182e-02          no     0
 89    1.139848e-02        more     1
 38    9.651138e-03          it     2
 988   8.615519e-03        yeah     3
 203   8.411843e-03        good     4
 ...            ...         ...   ...
 6964  5.794139e-07     annette  7899
 7334  5.395387e-07  yourselves  7900
 6494  5.354856e-07     frankly  7901
 7188  5.113905e-07    counters  7902
 5146  4.953320e-07       posed  7903
 
 [7904 rows x 3 columns])

In [249]:
# no context ft2: Fine-tuned BERT: with speaker labels
transfomers_bert_completions.get_stats_for_failure(
    all_tokens_phono, 16764425, ft2_bertMaskedLM, ft2_tokenizer,
    ft2_softmax_mask, None, use_speaker_labels=True)

[3.21968761e-03 2.89252523e-04 1.01761514e-04 ... 6.29579688e-07
 2.96900471e-05 3.03201432e-06]
['a' 'b' 'c' ... 'hideout' 'pudding' 'stalks']


/home/stephan/notebooks/child-directed-listening/lib/python3.7/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


(array([3.21968761e-03, 2.89252523e-04, 1.01761514e-04, ...,
        6.29579688e-07, 2.96900471e-05, 3.03201432e-06], dtype=float32),
               prob      word  rank
 56    1.858872e-01       one     0
 429   1.034543e-01     thing     1
 32    6.308804e-02        is     2
 51    2.303730e-02      this     3
 185   2.060782e-02      here     4
 ...            ...       ...   ...
 7634  3.872345e-09     lowry  7899
 6555  3.739657e-09   martins  7900
 7747  3.690924e-09  seasoned  7901
 7541  2.270597e-09   parades  7902
 7127  2.186263e-09      pubs  7903
 
 [7904 rows x 3 columns],
        rank  prob   entropy  num_tokens_in_context  bert_token_id
 26826   NaN   NaN  7.120152                      5          26826)

In [250]:
transfomers_bert_completions.get_stats_for_success(all_tokens_phono, 16759315, 
    ft2_bertMaskedLM, ft2_tokenizer, ft2_softmax_mask)

[1.02469595e-02 9.94500704e-03 8.72355234e-03 ... 5.15204647e-07
 1.10446425e-04 4.00284216e-06]
['a' 'b' 'c' ... 'hideout' 'pudding' 'stalks']


(array([[1.02469595e-02, 9.94500704e-03, 8.72355234e-03, ...,
         5.15204647e-07, 1.10446425e-04, 4.00284216e-06]], dtype=float32),
 [              prob        word  rank
  19    5.266874e-02           t     0
  18    2.626214e-02           s     1
  25    1.559725e-02         the     2
  7     1.444659e-02           h     3
  80    1.397641e-02          no     4
  ...            ...         ...   ...
  7630  2.317104e-07       racks  7899
  7398  2.180164e-07     nonstop  7900
  7396  2.086807e-07      hinges  7901
  7804  1.627102e-07       scuba  7902
  7334  1.254817e-07  yourselves  7903
  
  [7904 rows x 3 columns]],
     rank      prob   entropy  num_tokens_in_context  bert_token_id  \
 42   582  0.000242  9.730298                      1             42   
 
     mask_position  token  utterance_id  
 42              0  mommy      16759315  )

# BERT with Context

In [251]:
transfomers_bert_completions.bert_completions(
    "look ! [SEP] you see the ball . [SEP] what is it? [SEP] [MASK] !",
    adult_bertMaskedLM,
    adult_tokenizer,
    adult_softmax_mask
)

['[CLS]', 'look', '!', '[SEP]', 'you', 'see', 'the', 'ball', '.', '[SEP]', 'what', 'is', 'it', '?', '[SEP]', '[MASK]', '!', '[SEP]']
[2.9447689e-04 1.9055986e-05 9.5565183e-06 ... 2.8018499e-07 2.2896807e-06
 3.3121651e-07]
['a' 'b' 'c' ... 'hideout' 'pudding' 'stalks']


(array([2.9447689e-04, 1.9055986e-05, 9.5565183e-06, ..., 2.8018499e-07,
        2.2896807e-06, 3.3121651e-07], dtype=float32),
               prob       word  rank
 265   7.059029e-01       look     0
 1847  2.771682e-02        hey     1
 80    2.366572e-02         no     2
 630   2.194207e-02         oh     3
 73    2.063640e-02      there     4
 ...            ...        ...   ...
 6905  1.132056e-08   tempered  7899
 6420  1.007555e-08    prefers  7900
 4958  8.470574e-09     cooled  7901
 3372  7.183141e-09  addressed  7902
 7801  5.356558e-09   thanking  7903
 
 [7904 rows x 3 columns])

In [252]:
transfomers_bert_completions.bert_completions(
    "[MASK] !",
    adult_bertMaskedLM,
    adult_tokenizer,
    adult_softmax_mask
)

['[CLS]', '[MASK]', '!', '[SEP]']
[5.2099419e-04 1.9024061e-04 1.3560688e-04 ... 2.0262019e-06 4.0520654e-06
 5.2389805e-07]
['a' 'b' 'c' ... 'hideout' 'pudding' 'stalks']


(array([5.2099419e-04, 1.9024061e-04, 1.3560688e-04, ..., 2.0262019e-06,
        4.0520654e-06, 5.2389805e-07], dtype=float32),
               prob      word  rank
 180   1.421626e-01        go     0
 1847  1.083946e-01       hey     1
 336   2.970560e-02      help     2
 2042  2.592897e-02        ha     3
 6667  2.522623e-02     yahoo     4
 ...            ...       ...   ...
 6347  1.924245e-08   coating  7899
 7727  1.762578e-08    paving  7900
 4892  1.688352e-08   weighed  7901
 5142  9.923831e-09  repaired  7902
 5557  7.355099e-09  weighing  7903
 
 [7904 rows x 3 columns])

In [253]:
transfomers_bert_completions.bert_completions(
    "look ! [SEP] you see the ball . [SEP] what is it? [SEP] [MASK] !",
    ft1_bertMaskedLM,
    ft1_tokenizer, 
    ft1_softmax_mask
)

['[CLS]', 'look', '!', '[SEP]', 'you', 'see', 'the', 'ball', '.', '[SEP]', 'what', 'is', 'it', '?', '[SEP]', '[MASK]', '!', '[SEP]']
[3.3722995e-04 8.0243672e-06 1.3456638e-05 ... 2.2696716e-09 3.2982490e-07
 6.3708242e-08]
['a' 'b' 'c' ... 'hideout' 'pudding' 'stalks']


(array([3.3722995e-04, 8.0243672e-06, 1.3456638e-05, ..., 2.2696716e-09,
        3.2982490e-07, 6.3708242e-08], dtype=float32),
               prob       word  rank
 265   8.214665e-01       look     0
 1116  6.919919e-02       ball     1
 630   1.057040e-02         oh     2
 1002  8.318879e-03      watch     3
 163   4.932548e-03        see     4
 ...            ...        ...   ...
 7659  7.309568e-11    sockets  7899
 4542  6.387398e-11      petty  7900
 2645  6.073569e-11    greatly  7901
 2786  5.940642e-11  preferred  7902
 5559  3.651145e-11     vowels  7903
 
 [7904 rows x 3 columns])

In [254]:
transfomers_bert_completions.bert_completions(
    "[MASK] !",
    ft1_bertMaskedLM,
    ft1_tokenizer,
    ft1_softmax_mask
)

['[CLS]', '[MASK]', '!', '[SEP]']
[6.6175396e-03 9.3328045e-04 3.8436559e-04 ... 1.5088195e-06 2.5173698e-05
 8.6116580e-07]
['a' 'b' 'c' ... 'hideout' 'pudding' 'stalks']


(array([6.6175396e-03, 9.3328045e-04, 3.8436559e-04, ..., 1.5088195e-06,
        2.5173698e-05, 8.6116580e-07], dtype=float32),
               prob       word  rank
 180   5.293307e-02         go     0
 630   4.451400e-02         oh     1
 80    2.914032e-02         no     2
 2534  2.634110e-02         ah     3
 3567  2.466197e-02       boom     4
 ...            ...        ...   ...
 3585  2.473545e-08       isle  7899
 7659  1.701468e-08    sockets  7900
 4743  1.565106e-08    leisure  7901
 5189  1.538653e-08  greenwich  7902
 1929  1.454352e-08    assumed  7903
 
 [7904 rows x 3 columns])

In [255]:
transfomers_bert_completions.bert_completions(
    "[cgv] look ! [cgv] [SEP] you see the ball . [SEP] [cgv] what is it? [SEP] [cgv] [MASK] !",
    ft2_bertMaskedLM,
    ft2_tokenizer,
    ft2_softmax_mask
)

['[CLS]', '[cgv]', 'look', '!', '[cgv]', '[SEP]', 'you', 'see', 'the', 'ball', '.', '[SEP]', '[cgv]', 'what', 'is', 'it', '?', '[SEP]', '[cgv]', '[MASK]', '!', '[SEP]']
[1.3582357e-04 4.7675712e-06 8.8429761e-06 ... 9.6583508e-10 7.7967492e-08
 4.7733621e-08]
['a' 'b' 'c' ... 'hideout' 'pudding' 'stalks']


(array([1.3582357e-04, 4.7675712e-06, 8.8429761e-06, ..., 9.6583508e-10,
        7.7967492e-08, 4.7733621e-08], dtype=float32),
               prob     word  rank
 265   9.245384e-01     look     0
 1116  1.886555e-02     ball     1
 80    6.116604e-03       no     2
 630   5.484046e-03       oh     3
 163   4.926789e-03      see     4
 ...            ...      ...   ...
 7557  1.748249e-11     sima  7899
 6898  1.746130e-11  herring  7900
 7127  1.483380e-11     pubs  7901
 7659  1.388839e-11  sockets  7902
 6964  1.164501e-11  annette  7903
 
 [7904 rows x 3 columns])

In [256]:
full_context = "you can play it . [SEP] build the blocks . [SEP] look . [SEP] what is this [MASK] . [SEP] wee . [SEP] what ? [SEP] alright just a little . . ."
minimal_context = "what is this [MASK] ."

In [257]:
transfomers_bert_completions.bert_completions(full_context, adult_bertMaskedLM,
    adult_tokenizer, adult_softmax_mask)

['[CLS]', 'you', 'can', 'play', 'it', '.', '[SEP]', 'build', 'the', 'blocks', '.', '[SEP]', 'look', '.', '[SEP]', 'what', 'is', 'this', '[MASK]', '.', '[SEP]', 'wee', '.', '[SEP]', 'what', '?', '[SEP]', 'alright', 'just', 'a', 'little', '.', '.', '.', '[SEP]']
[4.1206350e-04 4.4401662e-05 1.1446742e-05 ... 3.1579634e-06 2.1047217e-05
 1.9096753e-07]
['a' 'b' 'c' ... 'hideout' 'pudding' 'stalks']


(array([4.1206350e-04, 4.4401662e-05, 1.1446742e-05, ..., 3.1579634e-06,
        2.1047217e-05, 1.9096753e-07], dtype=float32),
               prob       word  rank
 1849  1.846767e-01      stuff     0
 429   1.394682e-01      thing     1
 82    1.224868e-01      about     2
 1632  4.064887e-02       shit     3
 823   4.016055e-02        guy     4
 ...            ...        ...   ...
 4802  8.735933e-09     whites  7899
 3627  4.555039e-09    centres  7900
 7072  3.839028e-09    graders  7901
 4745  2.761369e-09     thirds  7902
 6726  2.212416e-09  crossings  7903
 
 [7904 rows x 3 columns])

In [258]:
transfomers_bert_completions.bert_completions(minimal_context, adult_bertMaskedLM,
    adult_tokenizer, adult_softmax_mask)

['[CLS]', 'what', 'is', 'this', '[MASK]', '.', '[SEP]']
[5.7512279e-05 4.6954942e-06 1.7185309e-06 ... 1.9388933e-06 7.3790943e-06
 2.9924431e-07]
['a' 'b' 'c' ... 'hideout' 'pudding' 'stalks']


(array([5.7512279e-05, 4.6954942e-06, 1.7185309e-06, ..., 1.9388933e-06,
        7.3790943e-06, 2.9924431e-07], dtype=float32),
               prob      word  rank
 179   7.037868e-01     place     0
 429   6.678923e-02     thing     1
 165   1.722324e-02       man     2
 823   1.432071e-02       guy     3
 5387  1.370183e-02  nonsense     4
 ...            ...       ...   ...
 6446  1.696233e-09  forwards  7899
 7072  1.578553e-09   graders  7900
 4302  1.565044e-09      laps  7901
 6001  1.560177e-09     wills  7902
 4745  8.996724e-10    thirds  7903
 
 [7904 rows x 3 columns])

In [259]:
transfomers_bert_completions.bert_completions(full_context, ft1_bertMaskedLM,
    ft1_tokenizer, ft1_softmax_mask)

['[CLS]', 'you', 'can', 'play', 'it', '.', '[SEP]', 'build', 'the', 'blocks', '.', '[SEP]', 'look', '.', '[SEP]', 'what', 'is', 'this', '[MASK]', '.', '[SEP]', 'wee', '.', '[SEP]', 'what', '?', '[SEP]', 'alright', 'just', 'a', 'little', '.', '.', '.', '[SEP]']
[4.8073740e-03 9.6353717e-05 2.3748234e-04 ... 1.4534255e-07 1.1587276e-05
 1.5726483e-06]
['a' 'b' 'c' ... 'hideout' 'pudding' 'stalks']


(array([4.8073740e-03, 9.6353717e-05, 2.3748234e-04, ..., 1.4534255e-07,
        1.1587276e-05, 1.5726483e-06], dtype=float32),
               prob      word  rank
 429   1.337474e-01     thing     0
 56    1.279441e-01       one     1
 185   4.643393e-02      here     2
 3809  4.023589e-02    fraser     3
 176   3.672646e-02    called     4
 ...            ...       ...   ...
 2645  1.122958e-09   greatly  7899
 7241  7.916627e-10   braking  7900
 6205  7.453299e-10     spurs  7901
 7866  4.192773e-10    condor  7902
 5985  2.194643e-10  turnpike  7903
 
 [7904 rows x 3 columns])

In [260]:
transfomers_bert_completions.bert_completions(minimal_context, ft1_bertMaskedLM,
    ft1_tokenizer, ft1_softmax_mask)

['[CLS]', 'what', 'is', 'this', '[MASK]', '.', '[SEP]']
[1.2034437e-02 9.0256333e-04 1.4439654e-04 ... 5.4023002e-07 2.6916116e-05
 6.8373361e-06]
['a' 'b' 'c' ... 'hideout' 'pudding' 'stalks']


(array([1.2034437e-02, 9.0256333e-04, 1.4439654e-04, ..., 5.4023002e-07,
        2.6916116e-05, 6.8373361e-06], dtype=float32),
               prob      word  rank
 56    9.787571e-02       one     0
 429   8.345397e-02     thing     1
 32    5.292996e-02        is     2
 185   4.436808e-02      here     3
 34    3.257630e-02       for     4
 ...            ...       ...   ...
 5126  6.021823e-09  airports  7899
 7241  6.009501e-09   braking  7900
 7866  3.241802e-09    condor  7901
 5583  2.310708e-09   motions  7902
 5679  1.696319e-09    leaked  7903
 
 [7904 rows x 3 columns])

In [261]:
# define minimal and full context with [cgv] and [chi] items
full_context = "[cgv] you can play it . [SEP] [cgv] build the blocks . [SEP] [cgv] look . [SEP] [cgv] what is this [MASK] . [SEP] [chi] wee . [SEP] [cgv] what ? [SEP] [cgv]  alright just a little . . ."
minimal_context = "[cgv] what is this [MASK] ."

In [262]:
transfomers_bert_completions.bert_completions(minimal_context, ft2_bertMaskedLM,
    ft2_tokenizer, ft2_softmax_mask)

['[CLS]', '[cgv]', 'what', 'is', 'this', '[MASK]', '.', '[SEP]']
[1.8574581e-03 2.2755688e-04 1.6224546e-04 ... 1.9756760e-07 2.6747892e-05
 4.2586885e-06]
['a' 'b' 'c' ... 'hideout' 'pudding' 'stalks']


(array([1.8574581e-03, 2.2755688e-04, 1.6224546e-04, ..., 1.9756760e-07,
        2.6747892e-05, 4.2586885e-06], dtype=float32),
               prob     word  rank
 56    7.246278e-02      one     0
 3880  6.886909e-02      huh     1
 185   5.969979e-02     here     2
 429   5.437483e-02    thing     3
 34    2.839113e-02      for     4
 ...            ...      ...   ...
 7100  2.665498e-09  boasted  7899
 7518  2.574831e-09  snooker  7900
 6205  2.376044e-09    spurs  7901
 7541  1.321016e-09  parades  7902
 7127  1.075959e-09     pubs  7903
 
 [7904 rows x 3 columns])

In [263]:
transfomers_bert_completions.bert_completions(full_context, ft2_bertMaskedLM,
    ft2_tokenizer, ft2_softmax_mask)

['[CLS]', '[cgv]', 'you', 'can', 'play', 'it', '.', '[SEP]', '[cgv]', 'build', 'the', 'blocks', '.', '[SEP]', '[cgv]', 'look', '.', '[SEP]', '[cgv]', 'what', 'is', 'this', '[MASK]', '.', '[SEP]', '[chi]', 'wee', '.', '[SEP]', '[cgv]', 'what', '?', '[SEP]', '[cgv]', 'alright', 'just', 'a', 'little', '.', '.', '.', '[SEP]']
[4.4540415e-04 1.3089532e-05 1.0229954e-04 ... 1.0216678e-07 1.2149234e-05
 3.1916500e-06]
['a' 'b' 'c' ... 'hideout' 'pudding' 'stalks']


(array([4.4540415e-04, 1.3089532e-05, 1.0229954e-04, ..., 1.0216678e-07,
        1.2149234e-05, 3.1916500e-06], dtype=float32),
               prob       word  rank
 176   1.486644e-01     called     0
 429   9.965955e-02      thing     1
 3853  9.522507e-02    darling     2
 185   9.165560e-02       here     3
 56    5.568583e-02        one     4
 ...            ...        ...   ...
 2617  6.267783e-10    credits  7899
 4581  5.322159e-10      fangs  7900
 6474  3.398692e-10  passports  7901
 6555  1.880796e-10    martins  7902
 7541  9.861655e-11    parades  7903
 
 [7904 rows x 3 columns])

# Successes and Failures

In [279]:
import transfomers_bert_completions
imp.reload(transfomers_bert_completions)

<module 'transfomers_bert_completions' from '/home/stephan/notebooks/child-directed-listening/transfomers_bert_completions.py'>

In [280]:
#### Here is a MWE to pick back up

transfomers_bert_completions.get_stats_for_failure(
    all_tokens_phono, 17280349, adult_bertMaskedLM, adult_tokenizer, adult_softmax_mask,
    5, use_speaker_labels=False)

/home/stephan/notebooks/child-directed-listening/lib/python3.7/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


IndexError: index out of range in self

In [293]:
# set the args
all_tokens = all_tokens_phono
selected_utt_id = 17280349
bertMaskedLM = adult_bertMaskedLM
tokenizer = adult_tokenizer
softmax_mask = adult_softmax_mask
context_width_in_utts = 2
use_speaker_labels =False

In [294]:
utt_df = all_tokens.loc[all_tokens.id == selected_utt_id]
utt_df.loc[utt_df.token == 'yyy','token'] = '[MASK]'
utt_df.loc[utt_df.token == '[MASK]','token_id'] = 103
utt_df[['token','token_id']]

/home/stephan/notebooks/child-directed-listening/lib/python3.7/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,token,token_id
3083434,[chi],30522
3083435,[MASK],103
3083436,.,1012


In [326]:
# check if yy is in the vocab
ft2_tokenizer.vocab["[chi]"]


KeyError: '[chi]'

In [317]:
len(ft2_tokenizer.vocab)

30522

In [ ]:
# Neither tokeinzer has xxx or yyy

In [295]:
this_transcript_id = utt_df.iloc[0].transcript_id

this_seq_utt_id = utt_df.iloc[0].seq_utt_id
before_utt_df = all_tokens.loc[(all_tokens.seq_utt_id < this_seq_utt_id) & (all_tokens.seq_utt_id > (this_seq_utt_id - (context_width_in_utts+1))) & (all_tokens['transcript_id'] == this_transcript_id)]

after_utt_df = all_tokens.loc[(all_tokens.seq_utt_id > this_seq_utt_id) & (all_tokens.seq_utt_id < this_seq_utt_id + (context_width_in_utts + 1)) & (all_tokens.transcript_id == this_transcript_id)]

# ready to use before_utt_df and after_utt_df    
sep_row = pd.DataFrame({'token':['[SEP]'], 'token_id':tokenizer.convert_tokens_to_ids(['[SEP]'])})

if before_utt_df.shape[0] > 0:
    before_by_sent = [x[1] for x in before_utt_df.groupby(['seq_utt_id'])]    
    i = n = 1
    while i < len(before_by_sent):
        before_by_sent.insert(i, sep_row)
        i += (n+1)
    before_by_sent_df = pd.concat(before_by_sent)
else:
    before_by_sent_df = pd.DataFrame()

if after_utt_df.shape[0] > 0:
    after_by_sent = [x[1] for x in after_utt_df.groupby(['seq_utt_id'])]    
    i = n = 1
    while i < len(after_by_sent):
        after_by_sent.insert(i, sep_row)
        i += (n+1)
    after_by_sent_df = pd.concat(after_by_sent)
else:
    after_by_sent_df = pd.DataFrame()

utt_df = pd.concat([before_by_sent_df, sep_row, utt_df, sep_row, after_by_sent_df])

# if preserve_errors:
#     #convert @ back to yyy for context items
#     utt_df.loc[utt_df.token == '@','token'] = 'yyy'
#     utt_df.loc[utt_df.token == 'yyy','token_id'] = tokenizer.convert_tokens_to_ids(['yyy'])[0]
#     #convert @ back to xxx for context items
#     utt_df.loc[utt_df.token == '$','token'] = 'xxx'
#     utt_df.loc[utt_df.token == 'xxx','token_id'] = tokenizer.convert_tokens_to_ids(['xxx'])[0]

if np.sum(utt_df.token == '[MASK]') > 1:
    print('Multiple masks in the surrounding context')
    import pdb
    pdb.set_trace() 

In [296]:
utt_df = utt_df.loc[~utt_df.token.isin(['[chi]','[cgv]'])][['token', 'token_id','bert_token_id']]
utt_df

,token,token_id,bert_token_id
3083429,yyy,30524,3083429.0
3083430,.,1012,3083430.0
0,[SEP],102,NaN
3083432,yyy,30524,3083432.0
3083433,.,1012,3083433.0
0,[SEP],102,NaN
3083435,[MASK],103,3083435.0
3083436,.,1012,3083436.0
0,[SEP],102,NaN
3083438,bee,10506,3083438.0


In [297]:
transfomers_bert_completions.get_completions_for_mask(utt_df, None, ft1_bertMaskedLM, ft1_tokenizer, softmax_mask)

IndexError: index out of range in self

In [219]:
transfomers_bert_completions.get_stats_for_success(all_tokens_phono, 16759315, adult_bertMaskedLM, 
        adult_tokenizer, adult_softmax_mask, 'score', None, use_speaker_labels=False, preserve_errors=True)

TypeError: get_stats_for_success() got multiple values for argument 'use_speaker_labels'

In [195]:
# no context, ft1: Fine-tuned BERT: don't use speaker labels
transfomers_bert_completions.get_stats_for_failure(
    all_tokens_phono, 17280276, ft1_bertMaskedLM, ft1_tokenizer, ft1_softmax_mask,
    5, use_speaker_labels=False, preserve_errors=True)

TypeError: get_stats_for_failure() got an unexpected keyword argument 'preserve_errors'

In [ ]:
# no context, ft1: Fine-tuned BERT: don't use speaker labels
transfomers_bert_completions.get_stats_for_success(all_tokens_phono, 16759315, ft1_bertMaskedLM, 
        ft1_tokenizer, ft1_softmax_mask, 'score', 5, use_speaker_labels=False, preserve_errors=True)

In [ ]:
# ft2: Fine-tuned BERT: with speaker labels
transfomers_bert_completions.get_stats_for_failure(
    all_tokens_phono, 17280276, ft2_bertMaskedLM, ft2_tokenizer,
    ft2_softmax_mask, 5, use_speaker_labels=True,
    preserve_errors=True)

In [194]:
# ft2: Fine-tuned BERT: with speaker labels
transfomers_bert_completions.get_stats_for_success(all_tokens_phono, 16759315, 
ft2_bertMaskedLM, ft2_tokenizer, ft2_softmax_mask, 5, use_speaker_labels=True)

IndexError: index out of range in self

In [3]:
### Retrieve from selected utterances

In [ ]:
# select 1000 success utterances and 1000 failure utterances
selected_success_utts = np.random.choice(success_utts.utterance_id, 500, replace=False)
selected_yyy_utts = np.random.choice(yyy_utts.utterance_id, 1000, replace=False)

# Unigram Model

In [ ]:
unigram_scores = transfomers_bert_completions.compare_successes_failures_unigram_model(
    all_tokens_phono,
    selected_success_utts,
    selected_yyy_utts,
    adult_tokenizer,
    adult_softmax_mask,
    'data/chi_vocab.csv',
    initial_vocab)

In [ ]:
unigram_scores_flat = transfomers_bert_completions.compare_successes_failures_unigram_model(
    all_tokens_phono,
    selected_success_utts,
    selected_yyy_utts,
    adult_tokenizer,
    adult_softmax_mask,
    None,
    initial_vocab)

In [11]:
transformers.__version__

'3.3.1'

# What is going on with "Each" probabilities?

In [92]:
from transformers import BertForMaskedLM as trans_BertForMaskedLM
from transformers import BertTokenizer as trans_BertTokenizer
 
wwm = trans_BertForMaskedLM.from_pretrained('bert-large-uncased-whole-word-masking')
wwm.eval()
wwm_tokenizer = trans_BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking')

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [98]:
wwm_softmax_mask, wwm_vocab = transfomers_bert_completions.get_softmax_mask(wwm_tokenizer, cmu_2syl_inchildes.word)
full_softmax_mask = np.argwhere(np.ones(len(wwm_tokenizer.vocab))).flatten()
full_softmax_mask

array([    0,     1,     2, ..., 30519, 30520, 30521])

In [99]:
completions = transfomers_bert_completions.bert_completions(
    "[MASK] nonfiction book has a call number on its spine .", wwm, 
    wwm_tokenizer, full_softmax_mask)

['[CLS]', '[MASK]', 'nonfiction', 'book', 'has', 'a', 'call', 'number', 'on', 'its', 'spine', '.', '[SEP]']


TypeError: tuple indices must be integers or slices, not tuple

### From the off-the shelf adult model

In [107]:
completions = transfomers_bert_completions.bert_completions(
    "[MASK] nonfiction book has a call number on its spine .", adult_bertMaskedLM, 
    adult_tokenizer, full_softmax_mask)

['[CLS]', '[MASK]', 'nonfiction', 'book', 'has', 'a', 'call', 'number', 'on', 'its', 'spine', '.', '[SEP]']


In [108]:
completions[1].head(10)

,prob,word,rank
2169,0.418243,each,0
2296,0.382689,every,1
1037,0.099194,a,2
1996,0.052032,the,3
2151,0.022856,any,4
2023,0.010374,this,5
2053,0.006768,no,6
2070,0.000921,some,7
2087,0.000679,most,8
2028,0.000631,one,9
